In [1]:
#necessary libraries
import re
import chardet 
import operator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud

from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D

from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
data = pd.read_csv('/content/trac2_iben_train.csv')

In [4]:
data_test = pd.read_csv('/content/trac2_iben_dev.csv')

In [5]:
data_test.head()

ID  ... Sub-task B
0  C59.2078  ...       NGEN
1   C68.537  ...       NGEN
2  C59.1344  ...       NGEN
3   C59.706  ...        GEN
4   C68.663  ...       NGEN

[5 rows x 4 columns]

In [6]:
data.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [7]:
data_test.drop(['ID','Sub-task B'], axis=1, inplace=True)

In [8]:
data.head()

Text Sub-task A
0                                      Dada taratari        NAG
1                              Tumi korbe Amar sathe        NAG
2  Ar ta chara a sob bessha peter cheleder okhan ...        OAG
3                   কাকের শরীরে ময়ুরের পাখা লাগিয়েছে        CAG
4  পতিতাদের চরিত্র রাজনৈতিক নেতাদের থেকে হাজার গু...        NAG

In [9]:
data['Sub-task A'].value_counts()

NAG    2078
CAG     898
OAG     850
Name: Sub-task A, dtype: int64

In [10]:
NAG_train = data[(data['Sub-task A']=='NAG')]
OAG_train = data[(data['Sub-task A']=='CAG')]
CAG_train = data[(data['Sub-task A']=='OAG')]

In [11]:
NAG = data_test[(data_test['Sub-task A']=='NAG')]
OAG = data_test[(data_test['Sub-task A']=='CAG')]
CAG = data_test[(data_test['Sub-task A']=='OAG')]

In [12]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG_train["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG_train["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG_train["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG_train.head()

Text  Sub-task A
0                                      Dada taratari           0
1                              Tumi korbe Amar sathe           0
4  পতিতাদের চরিত্র রাজনৈতিক নেতাদের থেকে হাজার গু...           0
5                                  Thik bolecho bhai           0
7                                             অমানুষ           0

In [13]:
import warnings as wrn
wrn.filterwarnings('ignore')

NAG["Sub-task A"] = 0 
#negatives_test["Sentiment"] = 0

OAG["Sub-task A"] = 1
#positives_test["Sentiment"] = 2

CAG["Sub-task A"] = 2
#neutrals_test["Sentiment"] = 1

NAG.head()

Text  Sub-task A
0   Ek dom sothik kotha bolecho jhekane theka uthe...           0
2                                    DARUN AKDOM THIK           0
5                                     Kadam thik dada           0
12                               Supar bolla cho dada           0
15                                              Pagle           0

In [14]:
data_train = pd.concat([NAG_train,
                  OAG_train,
                  CAG_train
                 ],axis=0)

In [15]:
data_test = pd.concat([NAG,
                  OAG,
                  CAG
                 ],axis=0)


In [16]:
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
#from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
## for bert language model
#import transformers

In [17]:
!python -m spacy download en_core_web_lg

     |████████████████████████████████| 827.9MB 1.2MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=926826873505b8372a88b6d2e95f2a0bcb263d716864e4c469666f29304453e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-9qorj5ze/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [18]:
import en_core_web_lg
nlp = en_core_web_lg.load()

In [19]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report

In [20]:
def get_vec(x):
  doc = nlp(x)
  vec = doc.vector
  return vec

In [21]:
data_train['vec'] = data_train['Text'].apply(lambda x: get_vec(x))

In [22]:
data_test['vec'] = data_test['Text'].apply(lambda x: get_vec(x))

In [23]:
X = data_train['vec'].to_numpy()
X = X.reshape(-1, 1)

In [24]:
P = data_test['vec'].to_numpy()
P = P.reshape(-1, 1)

In [25]:
X = np.concatenate(np.concatenate(X, axis = 0), axis = 0).reshape(-1, 300)

In [26]:
P = np.concatenate(np.concatenate(P, axis = 0), axis = 0).reshape(-1, 300)

In [27]:
y = data_train['Sub-task A']

In [28]:
Q = data_test['Sub-task A']

In [29]:
X_train = X
X_test = P
y_train = y 
y_test = Q



In [30]:
clf = LogisticRegression(solver='liblinear')

In [31]:
clf.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [32]:
y_pred = clf.predict(X_test)

In [33]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.61      0.94      0.74       522
           1       0.44      0.11      0.18       218
           2       0.68      0.32      0.44       217

    accuracy                           0.61       957
   macro avg       0.58      0.46      0.45       957
weighted avg       0.59      0.61      0.55       957

